#### 1.2 定义重量段

In [3]:
import os
import pandas as pd
import math
import glob

In [1]:
def Tier(weight):
    if weight > 10 :
        Tier = "X >10kg"
    elif weight > 5 :
        Tier = "X >5kg,≤10kg"
    elif weight > 3 :
        Tier = "X >3kg,≤5kg"
    elif weight > 2 :
        Tier = "X >2kg, ≤3kg"
    elif weight > 1 :
        Tier = "X >1kg, ≤2kg"
    elif weight > 0.5 :
        Tier = "X >0.5kg, ≤1kg"
    elif weight > 0.3 :
        Tier = "X >0.3kg, ≤0.5kg"
    else :
        Tier = "X ≤0.3kg"
    return Tier 

In [2]:
input_path = r"D:\我的坚果云\Work  Share\许程晨\5、Billing Summary\1.0 DL\2022年Summary"

In [7]:
content = []
for input_file in glob.glob(os.path.join(input_path,"*xlsx")):
    df = pd.read_excel(input_file)
    df["日期"] = os.path.basename(input_file)
    content.append(df)
order_handing = pd.concat(content)

In [15]:
order_handing[:5]

,运输类型,仓库区域,仓库名称,店铺/货主,品牌,订单类型,订单日期,发货日期,订单编号/包裹号,出库单号/销售订单编码,...,包装耗材-3,包装数量-3,包装单价-3,filename,805826.2899958541,日期,Unnamed: 47,重量段,计费重量,订单量
0,B2C-正向 (常规),太仓,康之诚太仓店铺仓B2C,京东pop-蜜纽康旗舰店,蜜纽康,销售订单,2022-01-27,2022-01-27,CK202201256887-1,CK202201270023,...,NaN,NaN,0.0,北京康之信、创新事业部、个护家清、红酒项目部、啤酒项目部、社交电商、天猫旗舰店 202201...,NaN,202201.xlsx,NaN,"X >0.5kg, ≤1kg",1,1
1,B2C-正向 (常规),太仓,康之诚太仓店铺仓B2C,京东pop-蜜纽康旗舰店,蜜纽康,销售订单,2022-01-27,2022-01-27,CK202201256888-1,CK202201270022,...,NaN,NaN,NaN,北京康之信、创新事业部、个护家清、红酒项目部、啤酒项目部、社交电商、天猫旗舰店 202201...,NaN,202201.xlsx,NaN,"X >0.5kg, ≤1kg",1,1
2,B2C-正向 (常规),太仓,康之诚太仓店铺仓B2C,拼多多-百岁山官方旗舰店,景田,销售订单,2022-01-27,2022-01-27,CK2022012616719-1,CK202201270021,...,NaN,NaN,NaN,北京康之信、创新事业部、个护家清、红酒项目部、啤酒项目部、社交电商、天猫旗舰店 202201...,NaN,202201.xlsx,NaN,X >10kg,17,1
3,B2C-正向 (常规),太仓,康之诚太仓店铺仓B2C,拼多多-百岁山官方旗舰店,景田,销售订单,2022-01-27,2022-01-27,CK202201271921,CK202201270020,...,NaN,NaN,NaN,北京康之信、创新事业部、个护家清、红酒项目部、啤酒项目部、社交电商、天猫旗舰店 202201...,NaN,202201.xlsx,NaN,X >10kg,17,0
4,B2C-正向 (常规),太仓,康之诚太仓店铺仓B2C,拼多多-百岁山官方旗舰店,景田,销售订单,2022-01-27,2022-01-27,CK2022012612370-1,CK202201270013,...,NaN,NaN,NaN,北京康之信、创新事业部、个护家清、红酒项目部、啤酒项目部、社交电商、天猫旗舰店 202201...,NaN,202201.xlsx,NaN,"X >5kg,≤10kg",10,1


##### 1.1、订单处理费

In [9]:
order_handing["重量段"] = order_handing["总重量 (kg)"].map(Tier)
order_handing["计费重量"] = order_handing["总重量 (kg)"].map(math.ceil)
order_handing["订单量"] = 1
order_handing["订单量"][order_handing["原始单号/外部订单号"].duplicated()==True] = 0

C:\Users\CHENGC~1\AppData\Local\Temp/ipykernel_800/31835873.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_handing["订单量"][order_handing["原始单号/外部订单号"].duplicated()==True] = 0


In [14]:
df_pivot_order_handing = order_handing.pivot_table(index=["运输类型","店铺/货主","仓库区域","重量段","品牌","日期"],
                          values=["物流单号/运单号","订单量","计费重量","总重量 (kg)","订单处理费","货品数量"],
                          aggfunc={"物流单号/运单号":"count","订单量":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","订单处理费":"sum",
                                  '货品数量':"sum"},margins=True)

df_pivot_order_handing
df_pivot_order_handing.rename(columns={"物流单号/运单号":"包裹量"},inplace=True)
df_pivot_order_handing = df_pivot_order_handing.reset_index()

df_pivot_order_handing

,运输类型,店铺/货主,仓库区域,重量段,品牌,日期,总重量 (kg),包裹量,计费重量,订单处理费,订单量,货品数量
0,B2B-正向 (猫超协同),够实惠平价超市,广州,X >10kg,哈尔滨,202201.xlsx,4.215440e+02,36,432,19.80,36,36
1,B2B-正向 (猫超协同),够实惠平价超市,广州,X >10kg,哈尔滨,202202.xlsx,9.338000e+01,8,96,4.40,7,8
2,B2B-正向 (猫超协同),够实惠平价超市,广州,X >10kg,哈尔滨,202203.xlsx,2.320000e+01,2,24,1.10,2,2
3,B2B-正向 (猫超协同),够实惠平价超市,广州,"X >3kg,≤5kg",刀唛,202201.xlsx,7.172000e+01,18,82,9.90,16,18
4,B2B-正向 (猫超协同),够实惠平价超市,广州,"X >3kg,≤5kg",刀唛,202202.xlsx,3.065000e+00,1,4,0.55,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1537,B2C-逆向 (顺丰),盒马鲜生,太仓,"X >5kg,≤10kg",百利,202202.xlsx,5.503000e+00,1,6,1.30,1,4
1538,B2C-逆向 (顺丰),福佳旗舰店,太仓,"X >3kg,≤5kg",福佳,202203.xlsx,9.540000e+00,3,12,2.73,3,3
1539,B2C-逆向 (顺丰),福佳旗舰店,太仓,X ≤0.3kg,福佳,202201.xlsx,2.850000e-01,1,1,0.91,1,1
1540,B2C-逆向 (顺丰),美玫牌旗舰店,太仓,X >10kg,美玫,202201.xlsx,1.212100e+01,1,13,1.30,1,7


In [18]:
output = r"D:\我的坚果云\Work  Share\许程晨\5、Billing Summary\8、VLC_2022"
df_pivot_order_handing.to_excel(os.path.join(output,"202201_202203_order_handing_dl.xlsx"))

##### 1.2、快递费

In [27]:
Last_Mile = order_handing.pivot_table(index=["日期","店铺/货主","品牌","仓库区域","省份","重量段","物流公司","运输类型"],
                          values=["物流单号/运单号","订单量","计费重量","总重量 (kg)","快递费","货品数量"],
                          aggfunc={"物流单号/运单号":"count","订单量":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","快递费":"sum",
                                  '货品数量':"sum"},margins=True)

Last_Mile

快递费  \
日期          店铺/货主    品牌  仓库区域 省份     重量段            物流公司 运输类型                     
202201.xlsx Mikes旗舰店 麦克斯 太仓   上海     X >0.5kg, ≤1kg 韵达快递 B2C-逆向 (常规)       0.00   
                                     X >2kg, ≤3kg   韵达快递 B2C-正向 (常规)       4.60   
                                     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      23.80   
                              云南省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      29.40   
                              内蒙古自治区 X >1kg, ≤2kg   顺丰快递 B2C-正向 (常规)      20.20   
...                                                                         ...   
202203.xlsx 酒水云集     斧头牌 太仓   福建省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      20.60   
                              重庆     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      26.20   
                              陕西省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      36.90   
            金像牌旗舰店   金像  太仓   湖南省    X >2kg, ≤3kg   韵达快递 B2C-逆向 (常规)       0.00   
All                                                                   784535.15   

                                                                         总重量 (kg)  \
日期          店铺/货主    品牌  仓库区域 省份     重量段            物流公司 运输类型                       
202201.xlsx Mikes旗舰店 麦克斯 太仓   上海     X >0.5kg, ≤1kg 韵达快递 B2C-逆向 (常规)       0.6340   
                                     X >2kg, ≤3kg   韵达快递 B2C-正向 (常规)       2.0540   
                                     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      14.3180   
                              云南省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      10.7960   
                              内蒙古自治区 X >1kg, ≤2kg   顺丰快递 B2C-正向 (常规)       1.2320   
...                                                                           ...   
202203.xlsx 酒水云集     斧头牌 太仓   福建省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)       9.0780   
                              重庆     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)       9.3420   
                              陕西省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      12.5000   
            金像牌旗舰店   金像  太仓   湖南省    X >2kg, ≤3kg   韵达快递 B2C-逆向 (常规)       2.2820   
All                                                                   346339.8185   

                                                                      物流单号/运单号  \
日期          店铺/货主    品牌  仓库区域 省份     重量段            物流公司 运输类型                    
202201.xlsx Mikes旗舰店 麦克斯 太仓   上海     X >0.5kg, ≤1kg 韵达快递 B2C-逆向 (常规)         1   
                                     X >2kg, ≤3kg   韵达快递 B2C-正向 (常规)         1   
                                     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)         4   
                              云南省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)         3   
                              内蒙古自治区 X >1kg, ≤2kg   顺丰快递 B2C-正向 (常规)         1   
...                                                                        ...   
202203.xlsx 酒水云集     斧头牌 太仓   福建省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)         2   
                              重庆     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)         2   
                              陕西省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)         3   
            金像牌旗舰店   金像  太仓   湖南省    X >2kg, ≤3kg   韵达快递 B2C-逆向 (常规)         1   
All                                                                      82367   

                                                                        计费重量  \
日期          店铺/货主    品牌  仓库区域 省份     重量段            物流公司 运输类型                  
202201.xlsx Mikes旗舰店 麦克斯 太仓   上海     X >0.5kg, ≤1kg 韵达快递 B2C-逆向 (常规)       1   
                                     X >2kg, ≤3kg   韵达快递 B2C-正向 (常规)       3   
                                     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      16   
                              云南省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      12   
                              内蒙古自治区 X >1kg, ≤2kg   顺丰快递 B2C-正向 (常规)       2   
...                                                                      ...   
202203.xlsx 酒水云集     斧头牌 太仓   福建省    X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      10   
                              重庆     X >3kg,≤5kg    韵达快递 B2C-正向 (常规)      10   
                       

In [28]:
Last_Mile.rename(columns={"物流单号/运单号":"包裹量"},inplace=True)

In [29]:
Last_Mile = Last_Mile.reset_index()

Last_Mile

,日期,店铺/货主,品牌,仓库区域,省份,重量段,物流公司,运输类型,快递费,总重量 (kg),包裹量,计费重量,订单量,货品数量
0,202201.xlsx,Mikes旗舰店,麦克斯,太仓,上海,"X >0.5kg, ≤1kg",韵达快递,B2C-逆向 (常规),0.00,0.6340,1,1,1,2
1,202201.xlsx,Mikes旗舰店,麦克斯,太仓,上海,"X >2kg, ≤3kg",韵达快递,B2C-正向 (常规),4.60,2.0540,1,3,1,1
2,202201.xlsx,Mikes旗舰店,麦克斯,太仓,上海,"X >3kg,≤5kg",韵达快递,B2C-正向 (常规),23.80,14.3180,4,16,3,4
3,202201.xlsx,Mikes旗舰店,麦克斯,太仓,云南省,"X >3kg,≤5kg",韵达快递,B2C-正向 (常规),29.40,10.7960,3,12,3,3
4,202201.xlsx,Mikes旗舰店,麦克斯,太仓,内蒙古自治区,"X >1kg, ≤2kg",顺丰快递,B2C-正向 (常规),20.20,1.2320,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,202203.xlsx,酒水云集,斧头牌,太仓,福建省,"X >3kg,≤5kg",韵达快递,B2C-正向 (常规),20.60,9.0780,2,10,1,2
7013,202203.xlsx,酒水云集,斧头牌,太仓,重庆,"X >3kg,≤5kg",韵达快递,B2C-正向 (常规),26.20,9.3420,2,10,1,2
7014,202203.xlsx,酒水云集,斧头牌,太仓,陕西省,"X >3kg,≤5kg",韵达快递,B2C-正向 (常规),36.90,12.5000,3,14,2,7
7015,202203.xlsx,金像牌旗舰店,金像,太仓,湖南省,"X >2kg, ≤3kg",韵达快递,B2C-逆向 (常规),0.00,2.2820,1,3,1,1


#### 1.5 输出pivot

In [30]:
output = r"D:\我的坚果云\Work  Share\许程晨\5、Billing Summary\8、VLC_2022"

In [31]:
Last_Mile.to_excel(os.path.join(output,"202201_202203_last_mile_dl.xlsx"))